Twitter sentiment analysis
data source - kaggle (we have installed kaggle library)

In [1]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: C:\Users\Manasi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


**Upload kaggle.json file**

In [2]:
import os
import sys
from pathlib import Path

# Get the path to the Kaggle executable
kaggle_path = Path(sys.executable).parent / "Scripts" / "kaggle.exe"

# Check if kaggle.exe exists at this path
if not kaggle_path.exists():
    raise FileNotFoundError(f"{kaggle_path} does not exist. Please check your installation.")

# Add the directory to the PATH environment variable
os.environ["PATH"] += os.pathsep + str(kaggle_path.parent)

# Verify if Kaggle is now recognized
!kaggle --version


Kaggle API 1.6.17


In [3]:
#Configuring the path of kaggle.json file
import os
import json
import shutil

# Create the .kaggle directory
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

# Define the source and destination paths
source = 'kaggle.json'  # Replace with the actual path to your kaggle.json file
destination = os.path.expanduser('~/.kaggle/kaggle.json')

# Copy the kaggle.json file to the .kaggle directory
shutil.copyfile(source, destination)

# Set the appropriate permissions for the kaggle.json file
os.chmod(destination, 0o600)

In [4]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
#Extracting compressed datset
from zipfile import ZipFile
dataset = 'sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
    zip.extractall()
    print("The dataset is extracted")

The dataset is extracted


Importing the necessary libraries

In [6]:
import numpy as np
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #Convert text to numerical data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data processing

In [7]:
#loading the data from csv file to pandas dataframe 
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [8]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [9]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


**since the original columns names are not read by the pd.read csv function
and the first column is shown as title column we need to solve this**

In [10]:
#naming the columns and reading the dataset againing

column_names = ['target', 'id','date','flag','user','text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [11]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
#checking the missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [13]:
#cheking the distribution of target columns 
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [14]:
#Converting target 4 to 1 --> indicationg positive tweet 
twitter_data.replace({'target': {4:1}}, inplace = True)

In [15]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

0 --> negative tweet
1 --> positive tweet

**Stemming**

In [16]:
#stemming --> processing of reducing a word to its root word
port_stem = PorterStemmer()   

In [17]:
pattern = re.compile('[^a-zA-Z]')

In [18]:
english_stopwords = stopwords.words('english')

In [19]:
def stemming(content):
    stemmed_content = re.sub(pattern,' ',content) #removing non alphabetic chars from the tweets
    stemmed_content = stemmed_content.lower() #converting upper case to lower case
    stemmed_content = stemmed_content.split() #converting the tweet into into list of words 
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in english_stopwords] #removing stop words before stemming
    stemmed_content = ' '.join(stemmed_content) #joining all the words in a tweet which we have stemmed

    return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [21]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


**Stemmed Data**

In [22]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


**Seperating the data and label out of the data set**

In [24]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [25]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [26]:
print(Y)

[0 0 0 ... 1 1 1]


**Splitting the data into training data and test data**

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state = 2 ) #stratify the target into positive and negative uniformly 

In [28]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [29]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [30]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


**Converting textual data to numerical data to be understabla by model**    
**Done using TFidf vectorizer**

In [31]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train) # all the training data will be taken and score will be given to each word
X_test = vectorizer.transform(X_test) 

In [32]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [33]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

**Training the Machine Learning Model --> Logistic Regression**

In [34]:
model = LogisticRegression(max_iter = 1000)

In [35]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

In [36]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [37]:
print("Accuracy score on the training data: ", training_data_accuracy)

Accuracy score on the training data:  0.79871953125


In [38]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [39]:
print("Accuracy score on the test data: ", test_data_accuracy)

Accuracy score on the test data:  0.77668125


**Saving the trained model**

In [40]:
import pickle

In [41]:
filename = 'twitter_trained_model.sav'
pickle.dump(model, open(filename, 'wb')) #new file will have the model

**Using the saved model for future prediction**

In [44]:
#Loading the saved model
loaded_model = pickle.load(open('twitter_trained_model.sav','rb'))

In [50]:
X_new = X_test[100]
print(Y_test[100])

0


In [51]:
prediction = loaded_model.predict(X_new)
print(prediction)

if prediction[0] == 0:
    print("It's a negative tweet")

else:
    print("It's a positive tweet")

[0]
It's a negative tweet
